In [11]:
import pandas as pd
import numpy as np

# Load the CSVs into pandas dataframes
df_1 = pd.read_csv('poi_nearest_streets_dummy_data.csv')  # First CSV with DISPLAY_LA, DISPLAY_LO
df_2 = pd.read_csv('OSMv2.csv')  # Second CSV with longitude, latitude
df_3 = pd.read_csv('cleaned_POIS_data.csv')   # Another CSV to match against


/var/folders/y1/4lqd5nbn40s5kll78htdr9480000gn/T/ipykernel_14271/358802258.py:7: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df_3 = pd.read_csv('cleaned_POIS_data.csv')   # Another CSV to match against


In [7]:
# Check the unique values in 'is_on_street' to understand its format
print(df_1['is_on_street'].unique())


[False  True]


In [12]:
df_filtered = df_1[df_1['is_on_street'] == True].copy()

# Normalize column names for join
df_filtered['HOUSE_NUMB'] = df_filtered['HOUSE_NUMB'].astype(str).str.strip()
df_filtered['STREET_NAM'] = df_filtered['STREET_NAM'].astype(str).str.strip()
df_2['house_number'] = df_2['house_number'].astype(str).str.strip()
df_2['street_name'] = df_2['street_name'].astype(str).str.strip()
df_3['HOUSE_NUMB'] = df_3['HOUSE_NUMB'].astype(str).str.strip()
df_3['STREET_NAM'] = df_3['STREET_NAM'].astype(str).str.strip()

# Merge with OSMv2
merged = df_filtered.merge(df_2, left_on=['HOUSE_NUMB', 'STREET_NAM'], right_on=['house_number', 'street_name'], how='inner')

# Merge with cleaned_POIS_data
merged = merged.merge(df_3[['HOUSE_NUMB', 'STREET_NAM', 'DISPLAY_LA', 'DISPLAY_LO']],
                      on=['HOUSE_NUMB', 'STREET_NAM'], how='inner', suffixes=('', '_df3'))

# Compute weighted coordinates
merged['weighted_latitude'] = 0.35 * merged['latitude'] + 0.65 * merged['DISPLAY_LA']
merged['weighted_longitude'] = 0.35 * merged['longitude'] + 0.65 * merged['DISPLAY_LO']

# Save output
output_df = merged[['HOUSE_NUMB', 'STREET_NAM', 'weighted_latitude', 'weighted_longitude']]
output_df.to_csv('weighted_output_coordinates.csv', index=False)


In [14]:
from rapidfuzz import fuzz
import pandas as pd

# Normalize function
def normalize(df, house_col, street_col):
    df[house_col] = df[house_col].astype(str).str.strip().str.lower()
    df[street_col] = df[street_col].astype(str).str.strip().str.lower()
    return df

# Normalize all DataFrames
df_filtered = df_1[df_1['is_on_street'] == True].copy()
df_filtered = normalize(df_filtered, 'HOUSE_NUMB', 'STREET_NAM')
df_2 = normalize(df_2, 'house_number', 'street_name')
df_3 = normalize(df_3, 'HOUSE_NUMB', 'STREET_NAM')

# Manual fuzzy matching loop
matches = []
for _, row in df_filtered.iterrows():
    best_score = 0
    best_match_row = None

    for _, cand_row in df_2.iterrows():
        score_house = fuzz.ratio(row['HOUSE_NUMB'], cand_row['house_number'])
        score_street = fuzz.ratio(row['STREET_NAM'], cand_row['street_name'])
        total_score = 0.3 * score_house + 0.7 * score_street

        if total_score > best_score:
            best_score = total_score
            best_match_row = cand_row

    if best_score >= 85:  # threshold
        combined = {**row.to_dict(), **best_match_row.to_dict()}
        matches.append(combined)

merged = pd.DataFrame(matches)

# Merge with df_3 (exact)
merged = merged.merge(df_3[['HOUSE_NUMB', 'STREET_NAM', 'DISPLAY_LA', 'DISPLAY_LO']],
                      on=['HOUSE_NUMB', 'STREET_NAM'], how='inner', suffixes=('', '_df3'))

# Weighted coordinates
merged['weighted_latitude'] = 0.35 * merged['latitude'] + 0.65 * merged['DISPLAY_LA']
merged['weighted_longitude'] = 0.35 * merged['longitude'] + 0.65 * merged['DISPLAY_LO']

# Save
output_df = merged[['HOUSE_NUMB', 'STREET_NAM', 'weighted_latitude', 'weighted_longitude']]
output_df.to_csv('weighted_output_coordinates.csv', index=False)
